<a href="https://colab.research.google.com/github/redinbluesky/the-lm-book/blob/main/03_순환_신경망.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 목차
* [Chapter 3_1 엘만 RNN](#chapter3_1)
* [Chapter 3_2 미니 배치 경사 하강법](#chapter3_2)

## Chapter 3_1 엘만 RNN <a class="anchor" id="chapter3_1"></a>
1. 순환 신경망(Recurrent Neural Network, RNN)은 순차 데이터(sequential data)를 위해 고안된 신경망이다.
    - 유닛 사이에 있는 연결에 루프가 포함되어 있어 시퀸스에서 한 단계의 정보가 다음 단계로 전달 될 수 있다.
    - 시계열 분석, 자연어 처리와 같은 분야에서 주로 사용된다.

2. 하나의 유닛을 가진 신경망과 입력 문서 "Learning from text is cool"이 있다고 가정하자.
    - 대소문자와 구두점을 무시하고 이 문서를 다음과 같은 행렬로 표현할 수 있다.
    <pre>
         단어     임베딩 벡터
         learning [0.1, 0.2, 0.6]
         from     [0.2, 0.1, 0.4]
         text     [0.1, 0.3, 0.5]
         is       [0.0, 0.7, 0.1]
         cool     [0.5, 0.2, 0.7]
         PAD      [0.0, 0.0, 0.0] (패딩 토큰)
    </pre>
    - 각 행은 신경망 훈련 과정에서 학습된 단어 임베딩 벡터이다.
    - 단어의 순서는 보존되며, 행렬 차원은 (시퀸스 길이, 임베딩차원) =(6, 3)이다.
       - 시퀸스 길이는 문서에 있는 최대 단어 수
    - 시퀸스 길이보다 짧은 문서는 패딩 토큰(PAD)으로 채운다.
       - 패딩 토큰은 더미 임베딩으로 일반적으로 0 벡터를 사용한다.
    - 수학적으로 행렬을 표현하면 아래와 같다.
    <pre>
    X = [ [0.1, 0.2, 0.6],
          [0.2, 0.1, 0.4],
          [0.1, 0.3, 0.5],
          [0.0, 0.7, 0.1],
          [0.5, 0.2, 0.7],
          [0.0, 0.0, 0.0] ]
    </pre>
    - 5개의 3차원 임베딩 벡터 x₁, x₂, x₃, x₄, x₅가 문서에 있는 단어를 나타낸다.
        - learning / x₁ = [0.1, 0.2, 0.6]ᵀ, from / x₂ = [0.2, 0.1, 0.4]ᵀ, text / x₃ = [0.1, 0.3, 0.5]ᵀ, 
        - is / x₄ = [0.0, 0.7, 0.1]ᵀ, cool / x₅ = [0.5, 0.2, 0.7]ᵀ

3. 엘만 RNN(Elman RNN)은 순환 신경망의 한 종류이다.
    - 아래의 그림과 같이 임베딩 벡터의 시퀸스를 한번에 하나씩 처리한다.
    - 각 타임 스텝 t에서 현재 입력 임베딩 벡터 xₜ와 이전 은닉 상태 hₜ₋₁를 훈련 가능한 가중치 Wₕ, Uₕ와 곱해진 후 편향 벡터 b와 더해져 은닉층 hₜ이 계산된다.
    - 스칼라를 출력하는 MLP 유닛과 달리 RNN 유닛은 벡터를 출력하고 하나의 층처럼 동작한다.
    - 초기 은닉상태 h₀는 일반적으로 0 벡터로 설정된다.

        ![Elman RNN](image/03-01-엘만_RNN.png)

    - 은닉 상태는 시퀸스에 있는 이전 단계의 정보를 기억하는 메모리 벡터이다.
    - 현재 입력과 지난 은닉 상태를 사용해 매 스탭마다 업데이트 되며, 신경망이 앞선 단어의 문맥을 사용해 다음 단어를 예측할 수 있도록 돕는다.
    - 심층 신경망을 만들려면 두 번째 RNN 층을 추가한다. 첫 번째 층의 출력이 두 번째 층의 입력이 된다.

        ![Stacked Elman RNN](image/03-01-스택_엘만_RNN.png)

## Chapter 3_2 미니 배치 경사 하강법 <a class="anchor" id="chapter3_2"></a>
1. 미니 배치 경사 하강법(Mini-batch Gradient Descent)은 신경망 훈련에 자주 사용되는 최적화 알고리즘이다.
    - 데이터의 작은 부분집합에서 도함수를 계산하여 학습 속도를 높이고 메모리 사용량을 줄인다.
    - 데이터 크기는 (미니 배치 크기, 시퀸스 길이, 임베딩 차원)의 형태가 된다.
    - 훈련 세트를 고정된 크기의 미니 배치로 나누고, 각 미니 배치에는 일정한 길이의 시퀸스가 포함된다.

2. 예를 들어 배치 크기 2, 시퀸스 길이 4, 임베딩 차원 3이라면 미니 배치는 다음과 같이 나타낼 수 있다.
    <pre>
    Batch = [ [seq1,1, seq1,2, seq1,3, seq1,4],
              [seq2,1, seq2,2, seq2,3, seq2,4] ]
    </pre>
    - seqi,j는 i∈{1,2}이고, j∈{1,2,3,4}인 임베딩 벡터이다.
    - 각 시퀸스의 임베딩 벡터가 다음과 같다고 가정한다.
    <pre>
    seq1 = [[0.1, 0.2, 0.3]
            [0.4, 0.5, 0.6]
            [0.7, 0.8, 0.9]
            [0.0, 1.1, 1.2]]
    seq2 = [[1.3, 1.4, 1.5]
            [1.6, 1.7, 1.8]    
            [1.9, 2.0, 2.1]
            [2.2, 2.3, 2.4]]
    </pre>
    - 이 경우 미니 배치는 다음과 같이 표현할 수 있다.
    <pre>
    Batch = [ [[0.1, 0.2, 0.3],
               [0.4, 0.5, 0.6],
               [0.7, 0.8, 0.9],
               [0.0, 1.1, 1.2]],    
              [[1.3, 1.4, 1.5],
               [1.6, 1.7, 1.8],    
               [1.9, 2.0, 2.1],
               [2.2, 2.3, 2.4]] ]
    </pre>
    - 미니 배치의 차원은 (2, 4, 3)이 된다.

3. 경사 하강법 단계마다 다음을 수행한다.
    - 훈련 세트에서 미니 배치를 추출한다.
    - 이를 신경망에 통과시킨다.
    - 손실을 계산한다.
    - 그레이디언트를 계산한다.
    - 모델 파라미터를 업데이트한다.
    - 이 과정을 훈련 세트 전체에 대해 반복한다.

4. 매 단계마다 전체 훈련 세트를 사용하는 경우보다 미니 배치 경사 하강법이 더 빠를게 수렴되는 경우가 많다.
    - 병렬처리 기능을 활용하여 대규모 모델과 데이터셋을 효율적으로 다룰 수 있다.
